#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [91]:
import pandas as pd
import pymongo

In [104]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['kickstarter']

In [110]:
collection.drop()

In [108]:
def parse_date(x) :
    try :
        return datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    except :
        return datetime.now()

In [111]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv", parse_dates=['launched'])
df_ks['launched'] = df_ks['launched'].apply(parse_date)

In [112]:
df_ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


In [113]:
df_ks.dtypes

ID                           int64
name                        object
category                    object
main_category               object
currency                    object
deadline                    object
goal                        object
launched            datetime64[ns]
pledged                     object
state                       object
backers                     object
country                     object
usd pledged                 object
usd_pledged_real           float64
dtype: object

Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [69]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [70]:
df_ks.isnull().sum()

ID                     0
name                   2
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         1482
usd_pledged_real       0
dtype: int64

### Importer les données

In [114]:
dict_donnees = df_ks.to_dict('records')

In [115]:
collection.insert_many(dict_donnees)

## Question 1  

Récupérer les 5 projets ayant reçu le plus de promesse de dons.

In [21]:
cur = collection.find({}).sort([("pledged", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('5fc4e26892f46015ba481912'),
  'ID': 218218259,
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-14',
  'goal': '7500.0',
  'launched': '2016-05-10 15:53:56',
  'pledged': '9996.0',
  'state': 'successful',
  'backers': '50',
  'country': 'US',
  'usd pledged': '9996.0',
  'usd_pledged_real': 9996.0},
 {'_id': ObjectId('5fc4e26892f46015ba47c7f0'),
  'ID': 1712352341,
  'name': 'Scott Davis - Solo Record',
  'category': 'Music',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2014-11-22',
  'goal': '8000.0',
  'launched': '2014-10-23 20:06:37',
  'pledged': '9995.0',
  'state': 'undefined',
  'backers': '0',
  'country': 'N,0"',
  'usd pledged': nan,
  'usd_pledged_real': 9995.0},
 {'_id': ObjectId('5fc4e26792f46015ba479fa0'),
  'ID': 1742823215,
  'name': 'Polco: A Digital Platform for Better Civic Communication',
  'category': 'Web',
  'ma

## Question 2

Compter le nombre de projets ayant atteint leur but.

In [23]:
collection.find({"state" : "successful"}).count(True)

<ipython-input-23-31eadd052d5d>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  collection.find({"state" : "successful"}).count(True)


53040

## Question 3

Compter le nombre de projets pour chaque catégorie.

In [29]:
df_ks['country'].unique()

array(['US', 'CA', 'HK', 'BE', 'GB', 'NO', 'NL', 'N,0"', 'AU', 'CH', 'SE',
       'AT', 'NZ', 'MX', 'FR', 'DK', 'IT', 'ES', 'DE', 'IE', 'SG', 'LU',
       'JP', '32'], dtype=object)

In [27]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "count" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Public Art', 'count': 1248},
 {'_id': 'Pet Fashion', 'count': 51},
 {'_id': 'Workshops', 'count': 59},
 {'_id': 'Digital Art', 'count': 524},
 {'_id': 'Places', 'count': 283},
 {'_id': 'Couture', 'count': 108},
 {'_id': 'Gadgets', 'count': 1210},
 {'_id': 'Web', 'count': 2017},
 {'_id': 'Blues', 'count': 113},
 {'_id': 'Candles', 'count': 168},
 {'_id': 'Farms', 'count': 482},
 {'_id': 'Plays', 'count': 539},
 {'_id': 'Horror', 'count': 525},
 {'_id': 'Installations', 'count': 178},
 {'_id': 'Space Exploration', 'count': 137},
 {'_id': 'Design', 'count': 1641},
 {'_id': 'Bacon', 'count': 78},
 {'_id': 'Printing', 'count': 83},
 {'_id': 'Cookbooks', 'count': 217},
 {'_id': 'Crafts', 'count': 1834},
 {'_id': 'Illustration', 'count': 1263},
 {'_id': 'Hip-Hop', 'count': 1555},
 {'_id': 'World Music', 'count': 850},
 {'_id': 'Art Books', 'count': 1065},
 {'_id': 'Jazz', 'count': 733},
 {'_id': 'Photo', 'count': 58},
 {'_id': 'Software', 'count': 1194},
 {'_id': 'Faith', 'count': 4

## Question 4

Compter le nombre de projets français ayant été instanciés avant 2016.

In [116]:
from datetime import datetime

cur = collection.find({"country":"FR", "launched": {"$lt" : datetime(2016, 1, 1, 0, 0, 0)}})
list(cur)

[{'_id': ObjectId('5fc5153592f46015ba4f6dbe'),
  'ID': 545482362,
  'name': 'Depeche Mode Bar Belgium',
  'category': 'Pop',
  'main_category': 'Music',
  'currency': 'EUR',
  'deadline': '2015-07-01',
  'goal': 20000.0,
  'launched': datetime.datetime(2015, 5, 27, 11, 29, 28),
  'pledged': 305.0,
  'state': 'failed',
  'backers': 10,
  'country': 'FR',
  'usd pledged': 333.98121895,
  'usd_pledged_real': 338.42637284599937},
 {'_id': ObjectId('5fc5153592f46015ba4f6f68'),
  'ID': 400288569,
  'name': 'Dracucat : A vampire kitten card game',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'EUR',
  'deadline': '2015-10-29',
  'goal': 1900.0,
  'launched': datetime.datetime(2015, 9, 29, 7, 4, 8),
  'pledged': 2944.0,
  'state': 'successful',
  'backers': 129,
  'country': 'FR',
  'usd pledged': 3295.37237632,
  'usd_pledged_real': 3243.3980764357875},
 {'_id': ObjectId('5fc5153592f46015ba4f7128'),
  'ID': 1990332786,
  'name': 'Minotaur enjoy your life & your ice

In [37]:
cur = collection.find({"country":"FR"})
list(cur)

[{'_id': ObjectId('5fc4e26492f46015ba464496'),
  'ID': 1131677007,
  'name': 'Bittersweet',
  'category': 'Shorts',
  'main_category': 'Film & Video',
  'currency': 'EUR',
  'deadline': '2016-02-04',
  'goal': 2000.0,
  'launched': '2016-01-05 06:00:18',
  'pledged': 2293.0,
  'state': 'successful',
  'backers': 44,
  'country': 'FR',
  'usd pledged': 2492.60262324,
  'usd_pledged_real': 2568.6120757253275},
 {'_id': ObjectId('5fc4e26492f46015ba4645fa'),
  'ID': 545482362,
  'name': 'Depeche Mode Bar Belgium',
  'category': 'Pop',
  'main_category': 'Music',
  'currency': 'EUR',
  'deadline': '2015-07-01',
  'goal': 20000.0,
  'launched': '2015-05-27 11:29:28',
  'pledged': 305.0,
  'state': 'failed',
  'backers': 10,
  'country': 'FR',
  'usd pledged': 333.98121895,
  'usd_pledged_real': 338.42637284599937},
 {'_id': ObjectId('5fc4e26492f46015ba464634'),
  'ID': 1442243866,
  'name': "GOB'Z'HEROES - The Game",
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': '

## Question 5

Récupérer les projets américains ayant demandé plus de 200 000 dollars.

In [51]:
cur = collection.find({"pledged":{"$gt": 200000}}, {"country":"US"})
list(cur)

[{'_id': ObjectId('5fc4e26492f46015ba4643ed'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba46447d'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba4644d8'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba46452b'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba464620'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba46464a'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba464659'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba46470e'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba464729'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba464741'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba4647b7'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba46480c'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba4648a0'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba4648f9'), 'country': 'US'},
 {'_id': ObjectId('5fc4e26492f46015ba46497e'), 'country': 'US'},
 {'_id': ObjectId('5fc4e2

## Question 6 

Compter le nombre de projet ayant "Sport" dans leur nom

In [50]:
collection.find({'name' : {'$regex':".*Sport.*"}}).count(True)


<ipython-input-50-40c84309e8a0>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  collection.find({'name' : {'$regex':".*Sport.*"}}).count(True)


323